In [6]:
from datetime import date

import pandas as pd
import time
import sys

# local modules
from binance.client import Client
from binance.enums import *
from indicator import indicators

# local file
import secrets
from configparser import ConfigParser
import yfinance as yf
import numpy as np
import datetime as dt
import requests 
import json 
import pandas as pd 
import numpy as np  
import logging

config=ConfigParser()
config.read('keys.txt')
import telepot
bot = telepot.Bot('1715056219:AAGxytb3U1gIt1vlVn8Jf5b4za3E1HPuOd4')
bot.getMe()

# %%
client = Client(str(config['KEYS']['binance_api_key']), str(config['KEYS']['binance_api_secret_key']))
logging.info("Logged in successfully")


# %%
position=""
logging.basicConfig(filename="logfilename.log", level=logging.INFO)



def candle(symbol, interval="5m"):


    root_url = 'https://api.binance.com/api/v1/klines'
    url = root_url + '?symbol=' + symbol + '&interval=' + interval
    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['Datetime',
                'Open', 'High', 'Low', 'Close', 'volume',
                'close_time', 'qav', 'num_trades',
                'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.close_time]
    
    df.drop(['close_time','qav','num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1,inplace=True)
           
    
    df['Open']=pd.to_numeric(df["Open"], downcast="float")
    df["High"]=pd.to_numeric(df["High"], downcast="float")
    df["Low"]=pd.to_numeric(df["Low"], downcast="float")
    df["Close"]=pd.to_numeric(df["Close"], downcast="float")
    df["volume"]=pd.to_numeric(df["volume"], downcast="float")
    
    
    
    
    df1=pd.DataFrame()
    Open=[]
    
    df1['Close']=((df['Open'] + df['High'] + df['Low'] + df['Close'])/4)
    for i in range(len(df)):
        if i==0:
            Open.append(0)

        else:
            Open.append((df['Open'][i-1]+df['Close'][i-1])/2)
    df1['Open']=np.array(Open)
    df1['volume']=df['volume']
    High=[]
    Low=[]
    High.append(0)
    Low.append(0)
    for i in range(1,len(df)):

        High.append(max(df['High'][i],df1['Close'][i],df1['Open'][i]))
        Low.append(min(df['Low'][i],df1['Close'][i],df1['Open'][i]))

    df1['High']=np.array(High)
    df1['Low']=np.array(Low)



    new_df = indicators.SuperTrend(df1, 5, 1)
    logging.info(new_df[:-1])
    return new_df


    

In [7]:
candle("BTCUSDT").tail(50)

TypeError: SMA() got an unexpected keyword argument 'alpha'

In [ ]:
while True:
    print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    print(candle("BTCUSDT").tail(50))